In [1]:
!wget -nc http://data.celltrackingchallenge.net/training-datasets/Fluo-C2DL-Huh7.zip
!unzip -n Fluo-C2DL-Huh7.zip

File ‘Fluo-C2DL-Huh7.zip’ already there; not retrieving.

Archive:  Fluo-C2DL-Huh7.zip


In [2]:
# !pip install stackview cellpose 'napari[all]' ultrack
# !pip install git+https://github.com/Janelia-Trackathon-2023/traccuracy
COLAB = True
# COLAB = False

In [3]:
from pathlib import Path

import numpy as np
import stackview
from dask.array.image import imread
from numpy.typing import ArrayLike
from rich import print

from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

from ultrack import track, to_tracks_layer, tracks_to_zarr, to_ctc
from ultrack.utils import labels_to_edges
from ultrack.config import MainConfig
from ultrack.imgproc import normalize
from ultrack.imgproc.segmentation import Cellpose
from ultrack.utils.array import array_apply

In [4]:
if COLAB:
    viewer = None
else:
    import napari
    viewer = napari.Viewer()

In [5]:
dataset = "02"
path = Path("Fluo-C2DL-Huh7") / dataset
image = imread(str(path / "*.tif"))

if COLAB:
    stackview.slice(image)
else:
    viewer.add_image(image)

In [6]:
cellpose = Cellpose(model_type="cyto2", gpu=True)

def predict(frame: ArrayLike, gamma: float) -> ArrayLike:
    norm_frame = normalize(np.asarray(frame), gamma=gamma)
    return cellpose(norm_frame, tile=False, normalize=False, diameter=75.0)

In [7]:
cellpose_labels = np.zeros(image.shape, dtype=np.int32)
array_apply(
    image,
    out_array=cellpose_labels,
    func=predict,
    gamma=0.5,
)

if COLAB:
    stackview.curtain(image, cellpose_labels)
else:
    viewer.add_labels(cellpose_labels)

Applying predict ...: 100%|████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:38<00:00,  5.27s/it]


In [8]:
detection, contours = labels_to_edges(cellpose_labels, sigma=5.0)

Converting labels to edges: 100%|██████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 15.61it/s]


In [9]:
if COLAB:
    stackview.curtain(image, detection)
else:
    viewer.add_labels(detection)

In [10]:
if COLAB:
    stackview.curtain(image, contours)
else:
    viewer.add_image(contours)

In [11]:
config = MainConfig()

# Candidate segmentation parameters
config.segmentation_config.n_workers = 8
config.segmentation_config.min_area = 2500
config.segmentation_config.min_frontier = 0.1

# Setting the maximum number of candidate neighbors and maximum spatial distance between cells
config.linking_config.max_neighbors = 5
config.linking_config.max_distance = 100
config.linking_config.n_workers = 8

# Adding absurd weight to division because there are few dividing cells
config.tracking_config.division_weight = -100

# Very few tracks enter/leave the field of view, increasing penalization
config.tracking_config.disappear_weight = -1
config.tracking_config.appear_weight = -1

print(config)

MainConfig(
    data_config=DataConfig(working_dir=PosixPath('.'), database='sqlite', address=None, n_workers=1),
    segmentation_config=SegmentationConfig(
        threshold=0.5,
        min_area=2500,
        max_area=1000000,
        min_frontier=0.1,
        anisotropy_penalization=0.0,
        max_noise=0.0,
        ws_hierarchy=<function watershed_hierarchy_by_area at 0x14315db40>,
        n_workers=8
    ),
    linking_config=LinkingConfig(
        n_workers=8,
        max_neighbors=5,
        max_distance=100,
        distance_weight=0.0,
        z_score_threshold=5.0
    ),
    tracking_config=TrackingConfig(
        appear_weight=-1,
        disappear_weight=-1,
        division_weight=-100,
        dismiss_weight_guess=None,
        include_weight_guess=None,
        window_size=None,
        overlap_size=1,
        solution_gap=0.001,
        time_limit=36000,
        method=0,
        n_threads=-1,
        link_function='power',
        power=4,
        bias=-0.0
    )
)

In [12]:
track(
    config,
    detection=detection,
    edges=contours,
    overwrite=True
)

Linking nodes.: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  8.50it/s]
ERROR:mip.cbc:An error occurred while loading the CBC library:	 cannot load library '/Users/jordao.bragantini/mambaforge/envs/ultrack/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib': dlopen(/Users/jordao.bragantini/mambaforge/envs/ultrack/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib, 0x0002): tried: '/Users/jordao.bragantini/mambaforge/envs/ultrack/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/jordao.bragantini/mambaforge/envs/ultrack/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (no such file), '/Users/jordao.bragantini/mambaforge/envs/ultrack/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, b

NameError: name 'cbclib' is not defined

In [ ]:
tracks_df, graph = to_tracks_layer(config)
tracks_df.to_csv(f"{dataset}_tracks.csv", index=False)

segments = tracks_to_zarr(
    config,
    tracks_df,
    overwrite=True,
)

if COLAB:
    stackview.curtain(image, segments)
else:
    viewer.add_tracks(
        tracks_df[["track_id", "t", "y", "x"]],
        name="tracks",
        graph=graph,
        visible=True,
    )
    
    viewer.add_labels(segments, name="segments").contour = 2

In [ ]:
name = f"{path.parent.name}_{path.name}".upper()
output_path = Path(name) / "TRA"
to_ctc(output_path, config, overwrite=True)

gt_path = path.parent / f"{dataset}_GT" / "TRA"

run_metrics(
    gt_data=load_ctc_data(gt_path), 
    pred_data=load_ctc_data(output_path),
    matcher=CTCMatched,
    metrics=[CTCMetrics],
)["CTCMetrics"]